In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from pathlib import Path
data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')

In [6]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s


def rle_decode(mask_rle, shape=(256, 256)):
    '''
    mask_rle: run-length as string formatted (start length)
              empty predictions need to be encoded with '-'
    shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''

    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    if mask_rle != '-': 
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
    return img.reshape(shape, order='F')  # Needed to align to RLE direction

In [7]:
r_bounds=(-4,2) #Refer RGB_Recipies PDF Page 7 
g_bounds=(-4,5)
b_bounds=(243,303)
def normalise(x,bounds):
    return (x-bounds[0])/(bounds[1]-bounds[0]) #Refer formula in PDF Pg no 3

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import *
from keras.layers import *
import numpy as np
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Input, Cropping2D, Flatten
from keras.models import Model
from tensorflow import keras
submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='record_id')
new_model=tf.keras.models.load_model("/kaggle/input/modelcnn/cnn_semantics.h5")
test_recs = os.listdir(data_path / 'test')
for rec in test_recs:
    band15=np.load(data_path / 'test' / rec / 'band_15.npy')
    band14=np.load(data_path / 'test' / rec / 'band_14.npy')
    band11=np.load(data_path / 'test' / rec / 'band_11.npy')
    r = normalise(band15 - band14, r_bounds)
    g = normalise(band14 - band11, g_bounds)
    b = normalise(band14, b_bounds)  
    false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    img=false_color[...,4]
    xt=np.expand_dims(img,axis=0)
    ypred=new_model.predict(xt,verbose=1)
    ypredt=ypred>0.5
    submission.loc[int(rec), 'encoded_pixels'] = list_to_string(rle_encode(ypredt))
    
submission.head()
    

1/1 [==============================] - 1s 1s/step


,encoded_pixels
record_id,
1000834164244036115,41222 3 41479 4 41736 5 41994 5 42252 5 42510 ...
1002653297254493116,-


In [10]:
submission.to_csv('submission.csv')